In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
import pickle as pk
import csv
import datetime
import lightgbm as lgb
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

#from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score
import re
import time
import warnings
warnings.filterwarnings('ignore')

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_X_tr = pd.read_csv('D:/transaction_risk_competition/transaction_train_new.csv')

In [3]:
train_X_tr=train_X_tr[['UID','day','time','trans_amt','bal','acc_id1','acc_id2','acc_id3','amt_src1','amt_src2','trans_type2','trans_type1','market_code','market_type','merchant','code1','code2','channel','device_code1','device_code2','device_code3','device1','device2','mac1','ip1','ip1_sub']]

In [4]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
#pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

train_X_tr.head(1)

,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type2,trans_type1,market_code,market_type,merchant,code1,code2,channel,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub
0,19092,30,11:48:47,1459,1007,544dfdeedad0ae67,NaN,NaN,acdbdb842ac20f1e,9fefed0a981dcb7a,102.0,26bcf43a19df14c8,2ecf94369847c748,1.0,abc6bc660561e0eb,NaN,NaN,102,ffda8eaa413171cb,b36097d8a79e5b43,NaN,d2cf44cec09806cc,VIVO X9L,7aaec0f23bfa60ea,a664530c1fb20abe,45f4ca1f4e9b9cb7


In [5]:
#统计各列字段的不完整性
#train_X_tr['acc_id1'].value_counts()#只计算非空值
#train_X_tr['acc_id1'].apply(lambda x :x/df['Prices'].sum())
#train_X_tr['acc_id1'].count()

# len_count=len(train_X_tr['UID'])
# for col in ['UID','day','time','trans_amt','bal','acc_id1','acc_id2','acc_id3','amt_src1','amt_src2','trans_type2','trans_type1','market_code','market_type','merchant','code1','code2','channel','device_code1','device_code2','device_code3','device1','device2','mac1','ip1','ip1_sub','geo_code']:
#     print('{0}'.format(col),  '%.2f' %(train_X_tr[col].isnull().sum()/len_count))

#需要因子化的列，
#train_X_tr['amt_src1'].value_counts()
#显示所有行
pd.set_option('display.max_rows', None)
# train_X_tr['geo_code'].value_counts()



In [6]:
#删除地理信息，目前不会挖掘
# train_X_tr.drop('geo_code',axis = 1,inplace = True)
# #train_X_tr.drop('acc_id1_isnull',axis = 1,inplace = True)
# train_X_tr.head(5)


In [7]:
#选取做因子化
##amt_src1 交易类型，余额，花呗做因子化
##trans_type2	线上线下 ，一定要因子化，只有4中，加上none,5中
##market_type	，营销类型，只有2种，加上空值，三种，一定要因子化
##channel 平台，渠道，5种，加上空值6种，适合因子化
values={'market_type':0}
train_X_tr.fillna(value=values,inplace = True)
train_X_tr.head(5)


,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type2,trans_type1,market_code,market_type,merchant,code1,code2,channel,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub
0,19092,30,11:48:47,1459,1007,544dfdeedad0ae67,NaN,NaN,acdbdb842ac20f1e,9fefed0a981dcb7a,102.0,26bcf43a19df14c8,2ecf94369847c748,1.0,abc6bc660561e0eb,NaN,NaN,102,ffda8eaa413171cb,b36097d8a79e5b43,NaN,d2cf44cec09806cc,VIVO X9L,7aaec0f23bfa60ea,a664530c1fb20abe,45f4ca1f4e9b9cb7
1,13465,23,16:58:03,725,100,NaN,NaN,NaN,4d7831c6f695ab19,NaN,105.0,c2f2023d279665b2,NaN,0.0,7a87b1aa20972d48,NaN,NaN,140,0e305dafdd3af261,365ec16ad4006c4f,NaN,09baf2f39bc3dc86,NEM-AL10,6f675a9b2124d2f8,52abe1e300a8aa10,2b416f1d7e89ad3b
2,13713,22,09:56:39,127,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,0.0,cfd035fc7e0a53b2,NaN,NaN,140,429d5656ba8f3d01,760cfbfc0c2e3b49,NaN,a92470e4f878a3f4,M3X,99a5f1ff7f3e54f0,e4c132d02ff6c095,ff71a556d86c4b8f
3,22703,1,18:11:45,99317,100,516e596377b2019c,NaN,NaN,a571c7fda8b7df37,fbf6bf3c8927414c,NaN,c2f2023d279665b2,NaN,0.0,c71c876b8979028e,NaN,NaN,140,609954adce541306,31989e264685a18c,NaN,09baf2f39bc3dc86,KNT-UL10,a8dc52f65085212e,e1c78a56cc9dfc8b,cf1527a0b4173e8f
4,17816,22,13:04:02,3361,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,0.0,6628f5ca013e3582,NaN,NaN,140,NaN,NaN,2f61e8f52151bd8b,aca4977fbe8741e1,IPHONE 5,NaN,abb6b4b8f15be6d6,cbc62c2b1cb7deb9


In [8]:
#对market_type因子化
train_X_tr['market_type'], uniques =pd.factorize(train_X_tr['market_type'],sort=True)
del uniques

In [9]:
# train_X_tr.head(5)
# columns=list(train_X_tr.columns)
# columns.remove('market_type')
# columns_del_market_type=columns
# train_X_tr=pd.get_dummies(train_X_tr['market_type'],prefix='market_type').join(train_X_tr[columns_del_market_type])
# train_X_tr.head(5)

In [10]:
values={'channel':0}
train_X_tr.fillna(value=values,inplace = True)

# columns=list(train_X_tr.columns)
# columns.remove('channel')
# columns_del_channel=columns
# train_X_tr=pd.get_dummies(train_X_tr['channel'],prefix='channel').join(train_X_tr[columns_del_channel])
# train_X_tr.head(5)


In [11]:
values={'trans_type2':0}
train_X_tr.fillna(value=values,inplace = True)

# columns=list(train_X_tr.columns)
# columns.remove('trans_type2')
# columns_del_trans_type2=columns
# train_X_tr=pd.get_dummies(train_X_tr['trans_type2'],prefix='trans_type2').join(train_X_tr[columns_del_trans_type2])
# train_X_tr.head(5)

In [12]:
values={'trans_type1':'0'}
train_X_tr.fillna(value=values,inplace = True)
train_X_tr['trans_type1'], uniques =pd.factorize(train_X_tr['trans_type1'],sort=True)
uniques
del uniques

# columns=list(train_X_tr.columns)
# #columns.remove('trans_type1')
# #columns_del_trans_type1=columns
# #最好还保留因子化后的trans_type1，放在后面的groupby字段里，同时get_dummies(train_X_tr['trans_type1']得到这笔交易是什么类型
# train_X_tr=pd.get_dummies(train_X_tr['trans_type1'],prefix='trans_type1').join(train_X_tr[columns])
# train_X_tr.head(5)

In [13]:
values={'amt_src1':'0'}
train_X_tr.fillna(value=values,inplace = True)

In [14]:
a=train_X_tr['amt_src1'].value_counts().index.tolist()
b=[0,1,2,3,4,5,6,7,8,9,10]+([-1]*(len(a)-11))
train_X_tr['amt_src1']=train_X_tr['amt_src1'].replace(a,b)
train_X_tr['amt_src1'].head(10)

0     7
1     2
2     0
3     5
4     0
5    -1
6    10
7     3
8     1
9     1
Name: amt_src1, dtype: int64

In [15]:
train_X_tr['merchant'].isnull().sum()/len(train_X_tr['UID'])#code1,code2,device_code3缺少太多，删掉这些列
train_X_tr=train_X_tr[['UID','day','time','trans_amt','bal','acc_id1','acc_id2','acc_id3','amt_src1','amt_src2','trans_type2','trans_type1','market_code','market_type','merchant','channel','device_code1','device_code2','device1','device2','mac1','ip1','ip1_sub']]

In [16]:
#train_X_tr.head(5)
#del train_X_tr_uid1
train_X_tr_uid1_count=train_X_tr[['acc_id1','acc_id2','acc_id3','amt_src1','amt_src2','trans_type2','trans_type1','market_code','market_type','merchant','channel','device_code1','device_code2','device1','device2','mac1','ip1','ip1_sub']].groupby(train_X_tr['UID']).count().add_suffix('_count').reset_index()
#train_X_tr_uid1.head(5)

In [17]:
train_X_tr_uid1_count.head(5)

,UID,acc_id1_count,acc_id2_count,acc_id3_count,amt_src1_count,amt_src2_count,trans_type2_count,trans_type1_count,market_code_count,market_type_count,merchant_count,channel_count,device_code1_count,device_code2_count,device1_count,device2_count,mac1_count,ip1_count,ip1_sub_count
0,10000,0,0,0,2,2,2,2,0,2,2,2,0,0,2,2,0,2,2
1,10001,14,1,1,18,14,18,18,3,18,18,18,16,16,16,16,16,18,18
2,10002,1,1,1,3,1,3,3,0,3,3,3,3,3,3,3,3,3,3
3,10003,3,3,3,4,3,4,4,0,4,4,4,4,4,4,4,4,4,4
4,10004,4,4,4,7,4,7,7,0,7,7,7,5,5,7,7,5,7,7


In [18]:
train_X_tr_uid1_count.shape

(30542, 19)

In [19]:
train_X_tr_uid2_channel_amt=train_X_tr.groupby(['UID','channel'])['trans_amt'].agg({'mean','max','min'}).add_prefix('amt_channel_').unstack()
train_X_tr_uid2_channel_amt.columns = [x[0]+"_"+str(x[1]) for x in train_X_tr_uid2_channel_amt.columns.ravel()]
train_X_tr_uid2_channel_amt.reset_index(inplace=True)
train_X_tr_uid2_channel_amt.fillna(0.0,inplace=True)
train_X_tr_uid2_channel_amt.head(5)



,UID,amt_channel_min_102,amt_channel_min_106,amt_channel_min_118,amt_channel_min_119,amt_channel_min_140,amt_channel_mean_102,amt_channel_mean_106,amt_channel_mean_118,amt_channel_mean_119,amt_channel_mean_140,amt_channel_max_102,amt_channel_max_106,amt_channel_max_118,amt_channel_max_119,amt_channel_max_140
0,10000,0.0,0.0,0.0,0.0,5536.0,0.000000,0.0,0.0,0.0,5536.00,0.0,0.0,0.0,0.0,5536.0
1,10001,1323.0,0.0,0.0,0.0,154.0,27929.642857,0.0,0.0,0.0,4448.75,108831.0,0.0,0.0,0.0,9287.0
2,10002,2818.0,0.0,0.0,0.0,1459.0,2818.000000,0.0,0.0,0.0,2138.50,2818.0,0.0,0.0,0.0,2818.0
3,10003,371.0,0.0,0.0,0.0,8254.0,4720.666667,0.0,0.0,0.0,8254.00,7983.0,0.0,0.0,0.0,8254.0
4,10004,5536.0,0.0,0.0,0.0,2818.0,22525.250000,0.0,0.0,0.0,3724.00,54465.0,0.0,0.0,0.0,5536.0


In [20]:
train_X_tr_uid2_channel_amt.shape

(30542, 16)

In [21]:
train_X_tr_uid2_market_type_amt=train_X_tr.groupby(['UID','market_type'])['trans_amt'].agg({'mean','max','min'}).add_prefix('amt_market_type_').unstack()
train_X_tr_uid2_market_type_amt.columns = [x[0]+"_"+str(x[1]) for x in train_X_tr_uid2_market_type_amt.columns.ravel()]
train_X_tr_uid2_market_type_amt.reset_index(inplace=True)
train_X_tr_uid2_market_type_amt.fillna(0.0,inplace=True)
train_X_tr_uid2_market_type_amt.head(5)

,UID,amt_market_type_min_0,amt_market_type_min_1,amt_market_type_min_2,amt_market_type_mean_0,amt_market_type_mean_1,amt_market_type_mean_2,amt_market_type_max_0,amt_market_type_max_1,amt_market_type_max_2
0,10000,5536.0,0.0,0.0,5536.000000,0.0,0.0,5536.0,0.0,0.0
1,10001,154.0,1323.0,0.0,26523.200000,3654.0,0.0,108831.0,8254.0,0.0
2,10002,1459.0,0.0,0.0,2365.000000,0.0,0.0,2818.0,0.0,0.0
3,10003,371.0,0.0,0.0,5604.000000,0.0,0.0,8254.0,0.0,0.0
4,10004,2818.0,0.0,0.0,14467.571429,0.0,0.0,54465.0,0.0,0.0


In [22]:
train_X_tr_uid2_market_type_amt.shape

(30542, 10)

In [23]:
train_X_tr_uid2_trans_type2_amt=train_X_tr.groupby(['UID','trans_type2'])['trans_amt'].agg({'mean','max','min'}).add_prefix('amt_trans_type2_').unstack()
train_X_tr_uid2_trans_type2_amt.columns = [x[0]+"_"+str(x[1]) for x in train_X_tr_uid2_trans_type2_amt.columns.ravel()]
train_X_tr_uid2_trans_type2_amt.reset_index(inplace=True)
train_X_tr_uid2_trans_type2_amt.fillna(0.0,inplace=True)
train_X_tr_uid2_trans_type2_amt.head(5)

,UID,amt_trans_type2_min_0.0,amt_trans_type2_min_102.0,amt_trans_type2_min_103.0,amt_trans_type2_min_104.0,amt_trans_type2_min_105.0,amt_trans_type2_mean_0.0,amt_trans_type2_mean_102.0,amt_trans_type2_mean_103.0,amt_trans_type2_mean_104.0,amt_trans_type2_mean_105.0,amt_trans_type2_max_0.0,amt_trans_type2_max_102.0,amt_trans_type2_max_103.0,amt_trans_type2_max_104.0,amt_trans_type2_max_105.0
0,10000,0.0,0.0,0.0,0.0,5536.0,0.0,0.000000,0.0,0.0,5536.00,0.0,0.0,0.0,0.0,5536.0
1,10001,0.0,1323.0,0.0,0.0,154.0,0.0,27929.642857,0.0,0.0,4448.75,0.0,108831.0,0.0,0.0,9287.0
2,10002,0.0,2818.0,0.0,0.0,1459.0,0.0,2818.000000,0.0,0.0,2138.50,0.0,2818.0,0.0,0.0,2818.0
3,10003,0.0,371.0,0.0,0.0,8254.0,0.0,4720.666667,0.0,0.0,8254.00,0.0,7983.0,0.0,0.0,8254.0
4,10004,0.0,5536.0,0.0,0.0,2818.0,0.0,22525.250000,0.0,0.0,3724.00,0.0,54465.0,0.0,0.0,5536.0


In [24]:
train_X_tr_uid2_trans_type2_amt.shape

(30542, 16)

In [25]:
train_X_tr_uid2_trans_type1_amt=train_X_tr.groupby(['UID','trans_type1'])['trans_amt'].agg({'mean','max','min'}).add_prefix('amt_trans_type1_').unstack()
train_X_tr_uid2_trans_type1_amt.columns = [x[0]+"_"+str(x[1]) for x in train_X_tr_uid2_trans_type1_amt.columns.ravel()]
train_X_tr_uid2_trans_type1_amt.reset_index(inplace=True)
train_X_tr_uid2_trans_type1_amt.fillna(0.0,inplace=True)
train_X_tr_uid2_trans_type1_amt.head(5)

,UID,amt_trans_type1_min_0,amt_trans_type1_min_1,amt_trans_type1_min_2,amt_trans_type1_min_3,amt_trans_type1_min_4,amt_trans_type1_min_5,amt_trans_type1_min_6,amt_trans_type1_min_7,amt_trans_type1_min_8,amt_trans_type1_min_9,amt_trans_type1_min_10,amt_trans_type1_min_11,amt_trans_type1_min_12,amt_trans_type1_min_13,amt_trans_type1_min_14,amt_trans_type1_mean_0,amt_trans_type1_mean_1,amt_trans_type1_mean_2,amt_trans_type1_mean_3,amt_trans_type1_mean_4,amt_trans_type1_mean_5,amt_trans_type1_mean_6,amt_trans_type1_mean_7,amt_trans_type1_mean_8,amt_trans_type1_mean_9,amt_trans_type1_mean_10,amt_trans_type1_mean_11,amt_trans_type1_mean_12,amt_trans_type1_mean_13,amt_trans_type1_mean_14,amt_trans_type1_max_0,amt_trans_type1_max_1,amt_trans_type1_max_2,amt_trans_type1_max_3,amt_trans_type1_max_4,amt_trans_type1_max_5,amt_trans_type1_max_6,amt_trans_type1_max_7,amt_trans_type1_max_8,amt_trans_type1_max_9,amt_trans_type1_max_10,amt_trans_type1_max_11,amt_trans_type1_max_12,amt_trans_type1_max_13,amt_trans_type1_max_14
0,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5536.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,5536.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5536.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,1323.0,0.0,0.0,0.0,10293.0,0.0,0.0,0.0,154.0,0.0,0.0,13628.0,0.0,0.0,0.0,3654.0,0.0,0.0,0.000000,36642.5,0.0,0.0,0.0,4448.75,0.0,0.0,13628.0,0.0,0.0,0.0,8254.0,0.0,0.0,0.0,108831.0,0.0,0.0,0.0,9287.0,0.0,0.0,13628.0,0.0,0.0,0.0
2,10002,0.0,0.0,0.0,2818.0,0.0,0.0,0.0,0.0,1459.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2818.000000,0.0,0.0,0.0,0.0,2138.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2818.0,0.0,0.0,0.0,0.0,2818.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10003,0.0,0.0,0.0,371.0,0.0,0.0,0.0,0.0,8254.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4720.666667,0.0,0.0,0.0,0.0,8254.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7983.0,0.0,0.0,0.0,0.0,8254.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10004,0.0,0.0,0.0,13691.0,0.0,0.0,0.0,0.0,2818.0,0.0,0.0,5536.0,0.0,0.0,0.0,0.0,0.0,0.0,34078.000000,0.0,0.0,0.0,0.0,3724.00,0.0,0.0,10972.5,0.0,0.0,0.0,0.0,0.0,0.0,54465.0,0.0,0.0,0.0,0.0,5536.0,0.0,0.0,16409.0,0.0,0.0,0.0


In [26]:
train_X_tr_uid2_trans_type1_amt.shape

(30542, 46)

In [27]:
train_X_tr_uid2_amt_src1_amt=train_X_tr.groupby(['UID','amt_src1'])['trans_amt'].agg({'mean','max','min'}).add_prefix('amt_amt_src1_').unstack()
train_X_tr_uid2_amt_src1_amt.columns = [x[0]+"_"+str(x[1]) for x in train_X_tr_uid2_amt_src1_amt.columns.ravel()]
train_X_tr_uid2_amt_src1_amt.reset_index(inplace=True)
train_X_tr_uid2_amt_src1_amt.fillna(0.0,inplace=True)
train_X_tr_uid2_amt_src1_amt.head(5)

,UID,amt_amt_src1_min_-1,amt_amt_src1_min_0,amt_amt_src1_min_1,amt_amt_src1_min_2,amt_amt_src1_min_3,amt_amt_src1_min_4,amt_amt_src1_min_5,amt_amt_src1_min_6,amt_amt_src1_min_7,amt_amt_src1_min_8,amt_amt_src1_min_9,amt_amt_src1_min_10,amt_amt_src1_mean_-1,amt_amt_src1_mean_0,amt_amt_src1_mean_1,amt_amt_src1_mean_2,amt_amt_src1_mean_3,amt_amt_src1_mean_4,amt_amt_src1_mean_5,amt_amt_src1_mean_6,amt_amt_src1_mean_7,amt_amt_src1_mean_8,amt_amt_src1_mean_9,amt_amt_src1_mean_10,amt_amt_src1_max_-1,amt_amt_src1_max_0,amt_amt_src1_max_1,amt_amt_src1_max_2,amt_amt_src1_max_3,amt_amt_src1_max_4,amt_amt_src1_max_5,amt_amt_src1_max_6,amt_amt_src1_max_7,amt_amt_src1_max_8,amt_amt_src1_max_9,amt_amt_src1_max_10
0,10000,0.0,0.0,0.0,0.0,5536.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,5536.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5536.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,0.0,154.0,10293.0,6977.0,0.0,0.0,0.0,81648.0,1323.0,0.0,0.0,0.0,0.0,765.5,21063.777778,8132.0,0.0,0.0,0.0,95239.5,3654.0,0.0,0.0,0.0,0.0,1377.0,81648.0,9287.0,0.0,0.0,0.0,108831.0,8254.0,0.0,0.0,0.0
2,10002,0.0,1459.0,2818.0,2818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1459.0,2818.000000,2818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1459.0,2818.0,2818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10003,0.0,0.0,371.0,8254.0,0.0,0.0,0.0,5808.0,0.0,0.0,0.0,0.0,0.0,0.0,4177.000000,8254.0,0.0,0.0,0.0,5808.0,0.0,0.0,0.0,0.0,0.0,0.0,7983.0,8254.0,0.0,0.0,0.0,5808.0,0.0,0.0,0.0,0.0
4,10004,0.0,2818.0,5536.0,2818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2818.0,22525.250000,4177.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2818.0,54465.0,5536.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
train_X_tr_uid2_amt_src1_amt.shape

(30542, 37)

In [29]:
train_X_tr_uid2_channel_bal = train_X_tr.groupby(['UID', 'channel'])['bal'].agg({'mean', 'max', 'min'}).add_prefix('bal_channel_').unstack()
train_X_tr_uid2_channel_bal.columns = [x[0] + "_" + str(x[1]) for x in train_X_tr_uid2_channel_bal.columns.ravel()]
train_X_tr_uid2_channel_bal.reset_index(inplace=True)
train_X_tr_uid2_channel_bal.fillna(0.0, inplace=True)
train_X_tr_uid2_channel_bal.head(5)

,UID,bal_channel_min_102,bal_channel_min_106,bal_channel_min_118,bal_channel_min_119,bal_channel_min_140,bal_channel_mean_102,bal_channel_mean_106,bal_channel_mean_118,bal_channel_mean_119,bal_channel_mean_140,bal_channel_max_102,bal_channel_max_106,bal_channel_max_118,bal_channel_max_119,bal_channel_max_140
0,10000,0.0,0.0,0.0,0.0,100.0,0.000000,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0
1,10001,100.0,0.0,0.0,0.0,100.0,62669.714286,0.0,0.0,0.0,100.0,156099.0,0.0,0.0,0.0,100.0
2,10002,100.0,0.0,0.0,0.0,100.0,100.000000,0.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,100.0
3,10003,100.0,0.0,0.0,0.0,100.0,100.000000,0.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,100.0
4,10004,100.0,0.0,0.0,0.0,100.0,100.000000,0.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,100.0


In [30]:
train_X_tr_uid2_channel_bal.shape

(30542, 16)

In [31]:
train_X_tr_uid2_market_type_bal = train_X_tr.groupby(['UID', 'market_type'])['bal'].agg({'mean', 'max', 'min'}).add_prefix('bal_market_type_').unstack()
train_X_tr_uid2_market_type_bal.columns = [x[0] + "_" + str(x[1]) for x in
train_X_tr_uid2_market_type_bal.columns.ravel()]
train_X_tr_uid2_market_type_bal.reset_index(inplace=True)
train_X_tr_uid2_market_type_bal.fillna(0.0, inplace=True)
train_X_tr_uid2_market_type_bal.head(5)

,UID,bal_market_type_min_0,bal_market_type_min_1,bal_market_type_min_2,bal_market_type_mean_0,bal_market_type_mean_1,bal_market_type_mean_2,bal_market_type_max_0,bal_market_type_max_1,bal_market_type_max_2
0,10000,100.0,0.0,0.0,100.000000,0.000000,0.0,100.0,0.0,0.0
1,10001,100.0,75853.0,0.0,37998.066667,102601.666667,0.0,156099.0,156099.0,0.0
2,10002,100.0,0.0,0.0,100.000000,0.000000,0.0,100.0,0.0,0.0
3,10003,100.0,0.0,0.0,100.000000,0.000000,0.0,100.0,0.0,0.0
4,10004,100.0,0.0,0.0,100.000000,0.000000,0.0,100.0,0.0,0.0


In [32]:
train_X_tr_uid2_market_type_bal.shape

(30542, 10)

In [33]:
train_X_tr_uid2_trans_type2_bal = train_X_tr.groupby(['UID', 'trans_type2'])['bal'].agg({'mean', 'max', 'min'}).add_prefix('bal_trans_type2_').unstack()
train_X_tr_uid2_trans_type2_bal.columns = [x[0] + "_" + str(x[1]) for x in train_X_tr_uid2_trans_type2_bal.columns.ravel()]
train_X_tr_uid2_trans_type2_bal.reset_index(inplace=True)
train_X_tr_uid2_trans_type2_bal.fillna(0.0, inplace=True)
train_X_tr_uid2_trans_type2_bal.head(5)

,UID,bal_trans_type2_min_0.0,bal_trans_type2_min_102.0,bal_trans_type2_min_103.0,bal_trans_type2_min_104.0,bal_trans_type2_min_105.0,bal_trans_type2_mean_0.0,bal_trans_type2_mean_102.0,bal_trans_type2_mean_103.0,bal_trans_type2_mean_104.0,bal_trans_type2_mean_105.0,bal_trans_type2_max_0.0,bal_trans_type2_max_102.0,bal_trans_type2_max_103.0,bal_trans_type2_max_104.0,bal_trans_type2_max_105.0
0,10000,0.0,0.0,0.0,0.0,100.0,0.0,0.000000,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0
1,10001,0.0,100.0,0.0,0.0,100.0,0.0,62669.714286,0.0,0.0,100.0,0.0,156099.0,0.0,0.0,100.0
2,10002,0.0,100.0,0.0,0.0,100.0,0.0,100.000000,0.0,0.0,100.0,0.0,100.0,0.0,0.0,100.0
3,10003,0.0,100.0,0.0,0.0,100.0,0.0,100.000000,0.0,0.0,100.0,0.0,100.0,0.0,0.0,100.0
4,10004,0.0,100.0,0.0,0.0,100.0,0.0,100.000000,0.0,0.0,100.0,0.0,100.0,0.0,0.0,100.0


In [34]:
train_X_tr_uid2_trans_type2_bal.shape

(30542, 16)

In [35]:
train_X_tr_uid2_trans_type1_bal = train_X_tr.groupby(['UID', 'trans_type1'])['bal'].agg({'mean', 'max', 'min'}).add_prefix('bal_trans_type1_').unstack()
train_X_tr_uid2_trans_type1_bal.columns = [x[0] + "_" + str(x[1]) for x in
train_X_tr_uid2_trans_type1_bal.columns.ravel()]
train_X_tr_uid2_trans_type1_bal.reset_index(inplace=True)
train_X_tr_uid2_trans_type1_bal.fillna(0.0, inplace=True)
train_X_tr_uid2_trans_type1_bal.head(5)

,UID,bal_trans_type1_min_0,bal_trans_type1_min_1,bal_trans_type1_min_2,bal_trans_type1_min_3,bal_trans_type1_min_4,bal_trans_type1_min_5,bal_trans_type1_min_6,bal_trans_type1_min_7,bal_trans_type1_min_8,bal_trans_type1_min_9,bal_trans_type1_min_10,bal_trans_type1_min_11,bal_trans_type1_min_12,bal_trans_type1_min_13,bal_trans_type1_min_14,bal_trans_type1_mean_0,bal_trans_type1_mean_1,bal_trans_type1_mean_2,bal_trans_type1_mean_3,bal_trans_type1_mean_4,bal_trans_type1_mean_5,bal_trans_type1_mean_6,bal_trans_type1_mean_7,bal_trans_type1_mean_8,bal_trans_type1_mean_9,bal_trans_type1_mean_10,bal_trans_type1_mean_11,bal_trans_type1_mean_12,bal_trans_type1_mean_13,bal_trans_type1_mean_14,bal_trans_type1_max_0,bal_trans_type1_max_1,bal_trans_type1_max_2,bal_trans_type1_max_3,bal_trans_type1_max_4,bal_trans_type1_max_5,bal_trans_type1_max_6,bal_trans_type1_max_7,bal_trans_type1_max_8,bal_trans_type1_max_9,bal_trans_type1_max_10,bal_trans_type1_max_11,bal_trans_type1_max_12,bal_trans_type1_max_13,bal_trans_type1_max_14
0,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,75853.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,100.0,0.0,0.0,0.0,102601.666667,0.0,0.0,0.0,56947.1,0.0,0.0,0.0,100.0,0.0,0.0,100.0,0.0,0.0,0.0,156099.0,0.0,0.0,0.0,156099.0,0.0,0.0,0.0,100.0,0.0,0.0,100.0,0.0,0.0,0.0
2,10002,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10003,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10004,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,100.0,0.0,0.0,0.0,0.000000,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,100.0,0.0,0.0,0.0


In [36]:
train_X_tr_uid2_trans_type1_bal.shape

(30542, 46)

In [37]:
train_X_tr_uid2_amt_src1_bal = train_X_tr.groupby(['UID', 'amt_src1'])['bal'].agg({'mean', 'max', 'min'}).add_prefix('amt_bal_src1_').unstack()
train_X_tr_uid2_amt_src1_bal.columns = [x[0] + "_" + str(x[1]) for x in train_X_tr_uid2_amt_src1_bal.columns.ravel()]
train_X_tr_uid2_amt_src1_bal.reset_index(inplace=True)
train_X_tr_uid2_amt_src1_bal.fillna(0.0, inplace=True)
train_X_tr_uid2_amt_src1_bal.head(5)

,UID,amt_bal_src1_min_-1,amt_bal_src1_min_0,amt_bal_src1_min_1,amt_bal_src1_min_2,amt_bal_src1_min_3,amt_bal_src1_min_4,amt_bal_src1_min_5,amt_bal_src1_min_6,amt_bal_src1_min_7,amt_bal_src1_min_8,amt_bal_src1_min_9,amt_bal_src1_min_10,amt_bal_src1_mean_-1,amt_bal_src1_mean_0,amt_bal_src1_mean_1,amt_bal_src1_mean_2,amt_bal_src1_mean_3,amt_bal_src1_mean_4,amt_bal_src1_mean_5,amt_bal_src1_mean_6,amt_bal_src1_mean_7,amt_bal_src1_mean_8,amt_bal_src1_mean_9,amt_bal_src1_mean_10,amt_bal_src1_max_-1,amt_bal_src1_max_0,amt_bal_src1_max_1,amt_bal_src1_max_2,amt_bal_src1_max_3,amt_bal_src1_max_4,amt_bal_src1_max_5,amt_bal_src1_max_6,amt_bal_src1_max_7,amt_bal_src1_max_8,amt_bal_src1_max_9,amt_bal_src1_max_10
0,10000,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,100.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,0.0,100.0,100.0,100.0,0.0,0.0,0.0,100.0,75853.0,0.0,0.0,0.0,0.0,100.0,63263.444444,100.0,0.0,0.0,0.0,100.0,102601.666667,0.0,0.0,0.0,0.0,100.0,156099.0,100.0,0.0,0.0,0.0,100.0,156099.0,0.0,0.0,0.0
2,10002,0.0,100.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.000000,100.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,100.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10003,0.0,0.0,100.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,100.000000,100.0,0.0,0.0,0.0,100.0,0.000000,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
4,10004,0.0,100.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.000000,100.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,100.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
train_X_tr_uid2_amt_src1_bal.shape

(30542, 37)

In [39]:
# train_X_tr_dt_mid=train_X_tr[['UID','day']].groupby(['UID']).mean().add_suffix("_mid").reset_index()
# train_X_tr=pd.merge(train_X_tr,train_X_tr_dt_mid,how='left',on='UID')
# train_X_tr.head(5)
##########只取近一半的数据，重新计数特征值作为新特征值

In [40]:
train_X_tr_uid_day = train_X_tr[['UID', 'day']]
train_X_tr_uid_day.drop_duplicates(['UID', 'day'], 'first', inplace=True)
train_X_tr_uid1_day_count = train_X_tr_uid_day[['day']].groupby(train_X_tr_uid_day['UID']).count().add_suffix('_count').reset_index()
train_X_tr_uid1_day_count.head(5)


,UID,day_count
0,10000,1
1,10001,9
2,10002,1
3,10003,1
4,10004,2


In [41]:
train_X_tr_uid1_day_count.shape

(30542, 2)

In [42]:
train_X_tr['day_time'] = train_X_tr.apply(lambda x: str(x['day']) + "_" + str(x['time']) , axis=1)
train_X_tr_uid1_day_time_count = train_X_tr[['day_time']].groupby(train_X_tr['UID']).count().add_suffix('_count').reset_index()
train_X_tr_uid1_day_time_count.head(5)


,UID,day_time_count
0,10000,2
1,10001,18
2,10002,3
3,10003,4
4,10004,7


In [43]:
train_X_tr_uid1_day_time_count.shape

(30542, 2)

In [44]:
train_X_tr_uid1_count_day_avg = pd.merge(train_X_tr_uid1_day_time_count,train_X_tr_uid1_day_count,how='left',on='UID')
train_X_tr_uid1_count_day_avg['count_day_avg']= train_X_tr_uid1_count_day_avg['day_time_count']/train_X_tr_uid1_count_day_avg['day_count']
train_X_tr_uid1_count_day_avg.head(5)

,UID,day_time_count,day_count,count_day_avg
0,10000,2,1,2.0
1,10001,18,9,2.0
2,10002,3,1,3.0
3,10003,4,1,4.0
4,10004,7,2,3.5


In [45]:
#train_X_tr_uid1_count_day_avg.shape

In [46]:
train_X_tr_uid1_count_day_avg.head(5)

,UID,day_time_count,day_count,count_day_avg
0,10000,2,1,2.0
1,10001,18,9,2.0
2,10002,3,1,3.0
3,10003,4,1,4.0
4,10004,7,2,3.5


In [47]:
#train_X_tr.loc[train_X_tr["UID"] == 10001]
#需要先排序
#train_X_tr['day'] = train_X_tr.groupby('UID')['day'].sort()
train_X_tr.sort_values(by=['UID','day','time'],inplace=True)
train_X_tr['day_shift'] = train_X_tr.groupby('UID')['day'].shift(-1)
train_X_tr.head(15)



,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type2,trans_type1,market_code,market_type,merchant,channel,device_code1,device_code2,device1,device2,mac1,ip1,ip1_sub,day_time,day_shift
90698,10000,26,12:23:56,5536,100,NaN,NaN,NaN,3,9a8ee16bde15e38a,105.0,8,NaN,0,88aa547576f43f85,140,NaN,NaN,aca4977fbe8741e1,IPHONE 5,NaN,34c715882de1a508,1e3ea9498c461cbf,26_12:23:56,26.0
134052,10000,26,12:24:17,5536,100,NaN,NaN,NaN,3,9a8ee16bde15e38a,105.0,8,NaN,0,88aa547576f43f85,140,NaN,NaN,aca4977fbe8741e1,IPHONE 5,NaN,34c715882de1a508,1e3ea9498c461cbf,26_12:24:17,NaN
63298,10001,2,08:31:13,9287,100,NaN,NaN,NaN,2,NaN,105.0,8,NaN,0,f3237791ad270d6b,140,e56819f72c9b7860,1aa9250077f98fea,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,2c1958d001a993d9,bd5e6a16e61e61f7,2_08:31:13,2.0
98298,10001,2,08:31:13,154,100,NaN,NaN,NaN,0,NaN,105.0,8,NaN,0,f3237791ad270d6b,140,e56819f72c9b7860,1aa9250077f98fea,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,2c1958d001a993d9,bd5e6a16e61e61f7,2_08:31:13,3.0
42602,10001,3,07:18:17,15186,61204,65407f8f309309a6,NaN,NaN,1,9fefed0a981dcb7a,102.0,4,NaN,0,a18cb3d84b9ec04c,102,e56819f72c9b7860,1aa9250077f98fea,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,bca1b486effef686,8e6d45472f232ed4,3_07:18:17,4.0
89982,10001,4,22:30:51,15186,46125,65407f8f309309a6,NaN,NaN,1,9fefed0a981dcb7a,102.0,4,NaN,0,a18cb3d84b9ec04c,102,e56819f72c9b7860,1aa9250077f98fea,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,85f4a2f59ceb4a1d,8f396a6bd85f9ba3,4_22:30:51,7.0
78949,10001,7,07:30:48,10293,22354,65407f8f309309a6,NaN,NaN,1,9fefed0a981dcb7a,102.0,4,NaN,0,a18cb3d84b9ec04c,102,e56819f72c9b7860,1aa9250077f98fea,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,39e2bf7f8f441d4f,509cdabae18d3c24,7_07:30:48,7.0
126895,10001,7,07:34:35,81648,100,3b60cd02636513ba,NaN,NaN,6,8d2ff85b6fd5dc78,102.0,4,NaN,0,3f6d3d0f42519ea4,102,e56819f72c9b7860,1aa9250077f98fea,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,39e2bf7f8f441d4f,509cdabae18d3c24,7_07:34:35,7.0
45908,10001,7,16:12:52,13628,100,65407f8f309309a6,65407f8f309309a6,4c287b179b08eb62,1,cf6e3a074407c379,102.0,11,NaN,0,3bd5cf7c40962299,102,e56819f72c9b7860,1aa9250077f98fea,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,c22ed313c546ab57,6d41ade4caf85310,7_16:12:52,8.0
23578,10001,8,07:49:23,15186,75290,65407f8f309309a6,NaN,NaN,1,9fefed0a981dcb7a,102.0,4,NaN,0,a18cb3d84b9ec04c,102,e56819f72c9b7860,1aa9250077f98fea,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,c22ed313c546ab57,6d41ade4caf85310,8_07:49:23,11.0


In [48]:
train_X_tr_uid_day_diff_avg=train_X_tr[['UID','day_shift']].groupby(['UID']).mean().add_suffix("_avg").reset_index()
train_X_tr_uid_day_diff_avg.fillna(0.0,inplace=True)
train_X_tr_uid_day_diff_avg.head(5)

,UID,day_shift_avg
0,10000,26.000000
1,10001,12.529412
2,10002,29.000000
3,10003,17.000000
4,10004,22.000000


In [49]:
train_X_tr_uid_day_diff_avg.shape


(30542, 2)

In [50]:
train_X_tr.columns

Index(['UID', 'day', 'time', 'trans_amt', 'bal', 'acc_id1', 'acc_id2',
       'acc_id3', 'amt_src1', 'amt_src2', 'trans_type2', 'trans_type1',
       'market_code', 'market_type', 'merchant', 'channel', 'device_code1',
       'device_code2', 'device1', 'device2', 'mac1', 'ip1', 'ip1_sub',
       'day_time', 'day_shift'],
      dtype='object')

In [51]:
train_X_tr_uid_day_diff_max=train_X_tr[['UID','day_shift']].groupby(['UID']).max().add_suffix("_max").reset_index()
train_X_tr_uid_day_diff_max.fillna(0.0,inplace=True)
train_X_tr_uid_day_diff_max.head(5)

,UID,day_shift_max
0,10000,26.0
1,10001,30.0
2,10002,29.0
3,10003,17.0
4,10004,29.0


In [52]:
train_X_tr_uid_day_diff_max.shape

(30542, 2)

In [53]:
train_X_tr_uid_day_diff_min=train_X_tr[['UID','day_shift']].groupby(['UID']).min().add_suffix("_min").reset_index()
train_X_tr_uid_day_diff_min.fillna(0.0,inplace=True)
train_X_tr_uid_day_diff_min.head(5)

,UID,day_shift_min
0,10000,26.0
1,10001,2.0
2,10002,29.0
3,10003,17.0
4,10004,8.0


In [54]:
train_X_tr_uid_day_diff_min.shape

(30542, 2)

In [55]:
# train_X_label= pd.read_csv('D:/transaction_risk_competition/tag_train_new.csv')
# train_X_label.shape

In [56]:
# train_X_label.columns

In [57]:
# train_X_label.Tag.value_counts(1)

In [58]:
# train_X_tr=pd.merge(train_X_tr_uid_day_diff_min,train_X_label,how='inner',on='UID')
# train_X_tr.shape

In [59]:
# train_X_tr.Tag.value_counts(1)

In [60]:
# train_X_or= pd.read_csv('D:/transaction_risk_competition/operation_train_new.csv')
# train_X_or.head(5)
#df1=train_X_or[['UID']].groupby(['UID'])
#df1.shape


In [61]:
# train_X_or_uid= train_X_or[['UID']]
# train_X_or_uid.drop_duplicates(['UID'], 'first', inplace=True)
# train_X_or_uid.reindex()
# train_X_or_uid.sort_values(by='UID').head(5)
# train_X_or_uid.shape

In [62]:
#train_X_uid_co = pd.merge(train_X_tr_uid_day_diff_min, train_X_or_uid, how='inner', on='UID')

In [63]:
#train_X_uid_co.shape

In [64]:
#train_X_uid_co.head(5)

In [65]:
#train_X_uid_co_label = pd.merge(train_X_uid_co, train_X_label, how='inner', on='UID')
#train_X_uid_co_label.shape

In [66]:
#train_X_uid_co_label.Tag.value_counts(1)

In [67]:
res_1 = pd.merge(train_X_tr_uid1_count_day_avg, train_X_tr_uid_day_diff_avg, how='left', on='UID')
res_1.shape


(30542, 5)

In [68]:
res_1 = pd.merge(res_1, train_X_tr_uid_day_diff_avg, how='left', on='UID')
res_1 = pd.merge(res_1, train_X_tr_uid_day_diff_min, how='left', on='UID')
res_1 = pd.merge(res_1, train_X_tr_uid_day_diff_max, how='left', on='UID')

########
res_1 = pd.merge(res_1, train_X_tr_uid2_channel_amt, how='left', on='UID')
res_1 = pd.merge(res_1, train_X_tr_uid2_market_type_amt, how='left', on='UID')
res_1 = pd.merge(res_1, train_X_tr_uid2_trans_type2_amt, how='left', on='UID')
res_1 = pd.merge(res_1, train_X_tr_uid2_trans_type1_amt, how='left', on='UID')
res_1 = pd.merge(res_1, train_X_tr_uid2_amt_src1_amt, how='left', on='UID')
########

res_1 = pd.merge(res_1, train_X_tr_uid2_channel_bal, how='left', on='UID')
res_1 = pd.merge(res_1, train_X_tr_uid2_market_type_bal, how='left', on='UID')
res_1 = pd.merge(res_1, train_X_tr_uid2_trans_type2_bal, how='left', on='UID')
res_1 = pd.merge(res_1, train_X_tr_uid2_trans_type1_bal, how='left', on='UID')
res_1 = pd.merge(res_1, train_X_tr_uid2_amt_src1_bal, how='left', on='UID')
########
res_1 = pd.merge(res_1, train_X_tr_uid1_count, how='left', on='UID')




NameError: name 'res' is not defined

In [69]:
res_1.shape

(30542, 266)

In [70]:
res_1.head(5)

,UID,day_time_count,day_count,count_day_avg,day_shift_avg_x,day_shift_avg_y,day_shift_min,day_shift_max,amt_channel_min_102,amt_channel_min_106,amt_channel_min_118,amt_channel_min_119,amt_channel_min_140,amt_channel_mean_102,amt_channel_mean_106,amt_channel_mean_118,amt_channel_mean_119,amt_channel_mean_140,amt_channel_max_102,amt_channel_max_106,amt_channel_max_118,amt_channel_max_119,amt_channel_max_140,amt_market_type_min_0,amt_market_type_min_1,amt_market_type_min_2,amt_market_type_mean_0,amt_market_type_mean_1,amt_market_type_mean_2,amt_market_type_max_0,amt_market_type_max_1,amt_market_type_max_2,amt_trans_type2_min_0.0,amt_trans_type2_min_102.0,amt_trans_type2_min_103.0,amt_trans_type2_min_104.0,amt_trans_type2_min_105.0,amt_trans_type2_mean_0.0,amt_trans_type2_mean_102.0,amt_trans_type2_mean_103.0,amt_trans_type2_mean_104.0,amt_trans_type2_mean_105.0,amt_trans_type2_max_0.0,amt_trans_type2_max_102.0,amt_trans_type2_max_103.0,amt_trans_type2_max_104.0,amt_trans_type2_max_105.0,amt_trans_type1_min_0,amt_trans_type1_min_1,amt_trans_type1_min_2,amt_trans_type1_min_3,amt_trans_type1_min_4,amt_trans_type1_min_5,amt_trans_type1_min_6,amt_trans_type1_min_7,amt_trans_type1_min_8,amt_trans_type1_min_9,amt_trans_type1_min_10,amt_trans_type1_min_11,amt_trans_type1_min_12,amt_trans_type1_min_13,amt_trans_type1_min_14,amt_trans_type1_mean_0,amt_trans_type1_mean_1,amt_trans_type1_mean_2,amt_trans_type1_mean_3,amt_trans_type1_mean_4,amt_trans_type1_mean_5,amt_trans_type1_mean_6,amt_trans_type1_mean_7,amt_trans_type1_mean_8,amt_trans_type1_mean_9,amt_trans_type1_mean_10,amt_trans_type1_mean_11,amt_trans_type1_mean_12,amt_trans_type1_mean_13,amt_trans_type1_mean_14,amt_trans_type1_max_0,amt_trans_type1_max_1,amt_trans_type1_max_2,amt_trans_type1_max_3,amt_trans_type1_max_4,amt_trans_type1_max_5,amt_trans_type1_max_6,amt_trans_type1_max_7,amt_trans_type1_max_8,amt_trans_type1_max_9,amt_trans_type1_max_10,amt_trans_type1_max_11,amt_trans_type1_max_12,amt_trans_type1_max_13,amt_trans_type1_max_14,amt_amt_src1_min_-1,amt_amt_src1_min_0,amt_amt_src1_min_1,amt_amt_src1_min_2,amt_amt_src1_min_3,amt_amt_src1_min_4,amt_amt_src1_min_5,amt_amt_src1_min_6,amt_amt_src1_min_7,amt_amt_src1_min_8,amt_amt_src1_min_9,amt_amt_src1_min_10,amt_amt_src1_mean_-1,amt_amt_src1_mean_0,amt_amt_src1_mean_1,amt_amt_src1_mean_2,amt_amt_src1_mean_3,amt_amt_src1_mean_4,amt_amt_src1_mean_5,amt_amt_src1_mean_6,amt_amt_src1_mean_7,amt_amt_src1_mean_8,amt_amt_src1_mean_9,amt_amt_src1_mean_10,amt_amt_src1_max_-1,amt_amt_src1_max_0,amt_amt_src1_max_1,amt_amt_src1_max_2,amt_amt_src1_max_3,amt_amt_src1_max_4,amt_amt_src1_max_5,amt_amt_src1_max_6,amt_amt_src1_max_7,amt_amt_src1_max_8,amt_amt_src1_max_9,amt_amt_src1_max_10,bal_channel_min_102,bal_channel_min_106,bal_channel_min_118,bal_channel_min_119,bal_channel_min_140,bal_channel_mean_102,bal_channel_mean_106,bal_channel_mean_118,bal_channel_mean_119,bal_channel_mean_140,bal_channel_max_102,bal_channel_max_106,bal_channel_max_118,bal_channel_max_119,bal_channel_max_140,bal_market_type_min_0,bal_market_type_min_1,bal_market_type_min_2,bal_market_type_mean_0,bal_market_type_mean_1,bal_market_type_mean_2,bal_market_type_max_0,bal_market_type_max_1,bal_market_type_max_2,bal_trans_type2_min_0.0,bal_trans_type2_min_102.0,bal_trans_type2_min_103.0,bal_trans_type2_min_104.0,bal_trans_type2_min_105.0,bal_trans_type2_mean_0.0,bal_trans_type2_mean_102.0,bal_trans_type2_mean_103.0,bal_trans_type2_mean_104.0,bal_trans_type2_mean_105.0,bal_trans_type2_max_0.0,bal_trans_type2_max_102.0,bal_trans_type2_max_103.0,bal_trans_type2_max_104.0,bal_trans_type2_max_105.0,bal_trans_type1_min_0,bal_trans_type1_min_1,bal_trans_type1_min_2,bal_trans_type1_min_3,bal_trans_type1_min_4,bal_trans_type1_min_5,bal_trans_type1_min_6,bal_trans_type1_min_7,bal_trans_type1_min_8,bal_trans_type1_min_9,bal_trans_type1_min_10,bal_trans_type1_min_11,bal_trans_type1_min_12,bal_trans_type1_min_13,bal_trans_type1_min_14,bal_trans_type1_mean_0,bal_trans_type1_mean_1,bal